In [1]:
import numpy as np
import math
import random
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

### Create a linear layer class in Keras

In [2]:
class Linear(keras.layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(units, ), initializer='zeros', trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [3]:
tf.ones((2,2))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
linear_layer = Linear(4,2)

In [5]:
linear_layer(tf.ones((2,2)))

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.05688385, 0.01943508, 0.11413171, 0.04054143],
       [0.05688385, 0.01943508, 0.11413171, 0.04054143]], dtype=float32)>

### Use build method to avoid having to specify input_dim

In [6]:
class Linear(keras.layers.Layer):
    
    def __init__(self, units=32):
        super().__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units, ), initializer='zeros', trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [7]:
class MLP(keras.layers.Layer):
    
    def __init__(self):
        super().__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)
        
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

In [8]:
#get maximum column value of each row

In [9]:
mlp = MLP()

In [10]:
y = mlp(tf.ones((2,2)))

In [11]:
y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.00410235, -0.00165037, -0.0029466 ,  0.00356041, -0.0011646 ,
         0.00081864,  0.00069409, -0.00141273, -0.00042577, -0.00399039],
       [-0.00410235, -0.00165037, -0.0029466 ,  0.00356041, -0.0011646 ,
         0.00081864,  0.00069409, -0.00141273, -0.00042577, -0.00399039]],
      dtype=float32)>

In [12]:
class ActivityRegularization(keras.layers.Layer):
    
    def __init__(self, rate=1e-2):
        super().__init__()
        self.rate = rate
    
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

In [13]:
tf.reduce_sum(tf.ones((14,10)))

<tf.Tensor: shape=(), dtype=float32, numpy=140.0>

In [14]:
mlp.losses

[]

In [15]:
class SparseMLP(keras.layers.Layer):
    
    def __init__(self):
        super().__init__()
        self.linear_1 = Linear(128)
        self.regularization = ActivityRegularization(1e-2)
        self.linear_3 = Linear(2)
    
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.regularization(x)
        return self.linear_3(x)

In [16]:
mlp = SparseMLP()
y = mlp(tf.ones((200, 4)))

In [17]:
mlp.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=9.975678>]

In [18]:
# write training loop for sparse mlp
# get dataset

# DQN
# collect transitions and go through network update process
# get max value of state-action pairs

In [19]:
(X_train, y_train), _ = keras.datasets.mnist.load_data()

In [20]:
dataset = tf.data.Dataset.from_tensor_slices(
    (X_train.reshape(60000, 784).astype('float32') / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

In [21]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [22]:
# access different elements of dataset
# get dataset of Transitions

In [23]:
linear_layer = Linear(10)

In [24]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

In [25]:
for step, (x, y) in enumerate(dataset):
    with tf.GradientTape() as tape:
        logits = linear_layer(x)
        loss = loss_fn(y, logits)

In [26]:
for step, (x, y) in enumerate(dataset):
    print(step, x, y)

0 tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 784), dtype=float32) tf.Tensor(
[9 1 3 5 4 3 2 8 0 5 6 8 7 8 4 9 0 7 0 5 7 6 9 5 1 5 0 0 2 4 5 0 8 8 7 1 4
 2 7 1 0 9 3 8 1 0 7 0 4 6 2 1 9 7 3 1 3 3 1 2 7 3 3 4], shape=(64,), dtype=uint8)
1 tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 784), dtype=float32) tf.Tensor(
[5 0 4 5 2 3 5 6 6 6 2 6 6 3 3 8 6 8 7 2 5 7 3 1 8 0 2 5 7 1 0 4 9 2 4 8 7
 4 6 1 9 2 2 8 7 4 4 4 7 2 7 2 1 1 7 9 8 4 7 8 6 8 9 0], shape=(64,), dtype=uint8)
2 tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 784), dtype=float32) tf.Tensor(
[4 9 8 2 6 5 7 4 2 1 4 7 0 5 7 7 5 5 6 6 7 3 

702 tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 784), dtype=float32) tf.Tensor(
[0 6 8 6 8 1 3 1 3 4 0 8 2 9 3 3 7 9 7 0 0 4 6 3 4 6 5 7 7 9 4 9 4 8 0 0 6
 2 3 9 3 8 2 9 3 0 2 2 6 8 6 3 4 1 8 9 6 1 7 6 3 2 2 9], shape=(64,), dtype=uint8)
703 tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 784), dtype=float32) tf.Tensor(
[0 7 7 2 0 6 5 3 0 9 4 3 7 3 8 0 7 5 9 2 3 1 3 8 7 5 5 8 6 2 6 6 6 8 1 5 1
 8 4 5 6 4 7 3 9 2 9 2 4 3 0 5 2 2 2 6 3 7 1 7 1 2 1 2], shape=(64,), dtype=uint8)
704 tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(64, 784), dtype=float32) tf.Tensor(
[6 7 6 5 1 0 2 5 0 6 8 5 1 7 6 0 5 8 3 

In [27]:
# how can you use this to create a Q-function? 

In [28]:
gradients = tape.gradient(loss, linear_layer.trainable_weights)

In [29]:
gradients

[<tf.Tensor: shape=(784, 10), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([-0.01450842, -0.01915271, -0.08951545,  0.15285331, -0.00610527,
         0.05841376, -0.00347064, -0.0668586 , -0.00274678, -0.00890919],
       dtype=float32)>]

In [30]:
optimizer.apply_gradients(zip(gradients, linear_layer.trainable_weights))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [31]:
tf.ones((5,12))

<tf.Tensor: shape=(5, 12), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [32]:
tf.random.normal((5, 12))

<tf.Tensor: shape=(5, 12), dtype=float32, numpy=
array([[-1.4676017 ,  0.27357918, -1.4880346 ,  0.41817227,  1.0565046 ,
        -0.39568418,  0.44012752, -0.36657634, -1.0373247 , -1.6504618 ,
         0.8796074 , -0.7933763 ],
       [-1.4406799 ,  0.60289156,  0.48947382,  0.83414376, -1.2527852 ,
         1.2447865 ,  0.8110041 ,  0.16195345, -0.7786184 ,  0.93952006,
        -0.96879977,  1.0509614 ],
       [-1.2925423 ,  0.35087913,  0.17723995,  0.6157899 ,  1.4859222 ,
        -2.3405733 , -0.25747296,  0.98789114,  1.2463928 , -0.85517085,
        -0.17676617, -1.7730851 ],
       [-2.3205087 , -0.7924822 ,  1.5283065 ,  1.2575012 ,  0.47105256,
        -1.9395502 , -0.6342582 , -0.1698841 , -0.07741747, -0.6085588 ,
        -1.4462225 ,  0.8922624 ],
       [-1.8159015 , -0.3397467 , -1.0628539 , -1.0606685 , -0.19651783,
         0.04843046,  1.0800045 ,  1.0524427 , -0.97740746, -0.26682907,
        -1.0861268 ,  1.0537477 ]], dtype=float32)>

In [33]:
# get array/tensor as input to model
tf.random.normal((303,5))

<tf.Tensor: shape=(303, 5), dtype=float32, numpy=
array([[-1.9453193 ,  0.77621233, -0.97086596, -0.9911053 , -1.2289379 ],
       [-0.9878419 , -0.8480158 , -0.19591586, -1.3940119 , -1.3852975 ],
       [ 0.48590285, -0.19464882,  1.3499566 ,  1.7746344 ,  0.9680634 ],
       ...,
       [-0.46702006,  0.53789175,  1.8025444 ,  0.45927846,  0.52341187],
       [ 2.0633943 ,  2.496735  , -0.88493824,  0.58040345, -0.39906517],
       [ 0.40138367, -1.5917861 , -0.7735407 , -1.1563163 ,  0.1899686 ]],
      dtype=float32)>

In [34]:
def get_basic_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [35]:
model = get_basic_model()

In [36]:
model.fit(tf.random.normal((303,5)), tf.random.normal((303,)), epochs=15, batch_size=128)

Epoch 1/15
3/3 [==============================] - 0s 2ms/step - loss: 0.7230 - accuracy: 0.0000e+00
Epoch 2/15
3/3 [==============================] - 0s 1ms/step - loss: 0.7063 - accuracy: 0.0000e+00
Epoch 3/15
3/3 [==============================] - 0s 1ms/step - loss: 0.6903 - accuracy: 0.0000e+00
Epoch 4/15
3/3 [==============================] - 0s 2ms/step - loss: 0.6740 - accuracy: 0.0000e+00
Epoch 5/15
3/3 [==============================] - 0s 2ms/step - loss: 0.6586 - accuracy: 0.0000e+00
Epoch 6/15
3/3 [==============================] - 0s 996us/step - loss: 0.6438 - accuracy: 0.0000e+00
Epoch 7/15
3/3 [==============================] - 0s 1ms/step - loss: 0.6293 - accuracy: 0.0000e+00
Epoch 8/15
3/3 [==============================] - 0s 1ms/step - loss: 0.6159 - accuracy: 0.0000e+00
Epoch 9/15
3/3 [==============================] - 0s 1ms/step - loss: 0.6011 - accuracy: 0.0000e+00
Epoch 10/15
3/3 [==============================] - 0s 1ms/step - loss: 0.5882 - accuracy: 0.0000e+

In [50]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

X = np.random.random((5000,22))
y = np.random.random((5000,1))
 
X_train,X_test, y_train,y_test = train_test_split(X,y)
 
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = dataset.shuffle(len(X_train)).batch(32)
train_data = train_data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
 
valid_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
valid_data = valid_ds.batch(32) 

def create_model():
    tfkl = tf.keras.layers
    inp = tf.keras.Input(shape=(2,))
    x = tfkl.Dense(128, activation="linear")(inp)
    x = tfkl.Dense(64, activation="linear")(x)
    x = tfkl.Dense(1, activation="linear")(x)
    
    model = tf.keras.models.Model(inp, x)
    model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    return model

model=create_model()
model.summary()

model.fit(train_data, epochs=3, validation_data=valid_data) 

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_15 (Dense)            (None, 128)               384       
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 8705 (34.00 KB)
Trainable params: 8705 (34.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3


ValueError: in user code:

    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users

In [53]:
def get_basic_model():
    model = tf.keras.Sequential([
    tf.keras.Input(shape=(2,)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

model = get_basic_model()
model.fit(tf.random.normal((300,22)), tf.random.normal((300,)), epochs=15, batch_size=128)

Epoch 1/15


ValueError: in user code:

    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa_environment\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users

In [40]:
x = tf.random.normal(shape=(2, 2))

with tf.GradientTape() as tape:
    tape.watch(x)
    y = tf.sin(x) 
    grads = tape.gradient(y, x)
    print(grads)

tf.Tensor(
[[0.5407251  0.86902046]
 [0.824367   0.17405969]], shape=(2, 2), dtype=float32)


In [41]:
tf.cos(x)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5407251 , 0.86902046],
       [0.824367  , 0.17405969]], dtype=float32)>

In [48]:
model.trainable_weights

[<tf.Variable 'dense_12/kernel:0' shape=(22, 10) dtype=float32, numpy=
 array([[ 0.18210363,  0.06558388, -0.19877401,  0.02729626, -0.06392241,
         -0.30586028, -0.21246454,  0.18012492, -0.31169233, -0.12754585],
        [ 0.22994632, -0.04889138,  0.03655186,  0.33452076,  0.24965326,
          0.01604195, -0.15614045, -0.02607657,  0.33787316,  0.1095162 ],
        [ 0.21906537, -0.12677705,  0.34867257, -0.31421578, -0.12555873,
         -0.01319539,  0.01558237, -0.09385934, -0.02489317, -0.37661713],
        [ 0.06544003, -0.109339  , -0.01801373, -0.3637322 ,  0.2878355 ,
          0.33068308,  0.04860697, -0.09196136, -0.29480702, -0.29094255],
        [ 0.2268069 ,  0.27416852, -0.3444399 , -0.09865692,  0.08021609,
         -0.3217839 ,  0.22919917, -0.18297367, -0.03635162,  0.12634149],
        [ 0.31019393,  0.23518737, -0.30746323,  0.3242955 , -0.25878897,
         -0.22188647,  0.17766534, -0.36163244, -0.32194623, -0.20910825],
        [-0.03881628, -0.28854176, 

In [ ]:

with tf.GradientTape() as tape:
    tape.watch(x)
    y = tf.sin(x) 
    grads = tape.gradient(y, x)
    print(grads)

In [49]:
loss_function = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

x = tf.random.normal(shape=(2, 2))
y = tf.sin(x) 

for i in range(100):
    with tf.GradientTape() as tape:
        tape.watch(x)
        y_pred = model(x)
        loss = loss_function(y, y_pred)
        grads = tape.gradient(loss, model.trainable_weights) # what to use here to get correct weights
        print(grads)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

ValueError: Input 0 of layer "sequential_3" is incompatible with the layer: expected shape=(None, 22), found shape=(2, 2)

In [59]:
class SparseMLP(keras.layers.Layer):
    """Stack of Linear layers with a sparsity regularization loss."""

    def __init__(self):
        super().__init__()
        self.linear_1 = Linear(32)
        self.regularization = ActivityRegularization(1e-2)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.regularization(x)
        return self.linear_3(x)


model = SparseMLP()
y = model(tf.ones((10, 10)))

print(model.losses)  # List containing one float32 scalar

(x_train, y_train), _ = keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.22144365>]


In [60]:
model.trainable_weights

[<tf.Variable 'sparse_mlp_3/linear_11/Variable:0' shape=(10, 32) dtype=float32, numpy=
 array([[ 0.12030625, -0.00103964, -0.06604697, -0.06918532, -0.08501961,
         -0.04875679,  0.00281884,  0.02390451, -0.04571602, -0.04901844,
         -0.02874516, -0.03204491,  0.08449087,  0.03347891,  0.00992027,
         -0.01522751, -0.04579986,  0.0243241 , -0.0077487 ,  0.01627476,
          0.01633072,  0.06231571, -0.03726816,  0.00245358, -0.04133619,
          0.0973319 , -0.04828577, -0.04703921,  0.01151996,  0.00949634,
         -0.06594551, -0.07115407],
        [ 0.04136331,  0.04989912,  0.02381083, -0.03362105,  0.04431029,
          0.01166568, -0.08920907, -0.01539674, -0.03071718,  0.08490882,
          0.00807447,  0.00596494, -0.00346296,  0.00625272,  0.05766921,
         -0.05581943,  0.05136488,  0.04024389, -0.06201141,  0.04294226,
         -0.02580767,  0.0410495 ,  0.017796  ,  0.05726192,  0.01802521,
         -0.03072983,  0.00739603, -0.05230518,  0.14949296, -0

In [63]:
loss_function = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

x = tf.random.normal(shape=(2, 2))
y = tf.sin(x) 

for i in dataset:
    with tf.GradientTape() as tape:
        tape.watch(x)
        y_pred = model(x)
        loss = loss_function(y, y_pred)
        grads = tape.gradient(loss, model.trainable_weights) # what to use here to get correct weights
        print(grads)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

InvalidArgumentError: Exception encountered when calling layer 'linear_11' (type Linear).

{{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [2,2], In[1]: [10,32] [Op:MatMul] name: 

Call arguments received by layer 'linear_11' (type Linear):
  • inputs=tf.Tensor(shape=(2, 2), dtype=float32)